# Análise dos dados de periódicos da Hemerotéca Digital Brasileira da Biblioteca Nacional

Os dados foram disponibilizados para pesquisa pela equipe da BNDigital em formato xml. O arquivo foi exportado da base do sistema de periódicos digitalizados e se encontra no padrão MARCXML (MAchine-Readable Cataloging XML). O arquivo pode ser acessado [aqui](/repositorios/BND-BR/data/exp_per_marcxml.xml).

Agradeço a Vinicius Pontes Martins do Setor de Gestão de Programas e Inovação (SGPI) da BNDigital pela disponibilização dos dados.

## Analisando e Filtrando o xml

Aqui vamos realizar realizar o *parse* dos dados contidos no arquivo xml e filtrar os dados que serão utilizados para análise.

Utilizaremos a biblioteca xml.etree.ElementTree para realizar o *parse* do arquivo xml. A biblioteca é nativa do Python e não necessita de instalação.

In [1]:
# importar bibliotecas
import xml.etree.ElementTree as ET

In [2]:
# ler arquivo xml
tree = ET.parse('./data/exp_per_marcxml.xml')

In [3]:
# contar número de registros
root = tree.getroot()
print(len(root))

7685


### Subcampos utilizados

Após a leitura do arquivos, selecionei sete subcampos para análise (serão listadas a tag e o subcampo de acordo com a estrutura do xml):

- tag 245; subcampo a: Título do periódico
- tag 245; subcampo b: Subtítulo do periódico
- tag 260; subcampo a: Local de publicação
- tag 260; subcampo b: Editora
- tag 260; subcampo c: Período de publicação
- tag 310; subcampo a: Periodicidade da publicação
- tag 546; subcampo a: Idioma da publicação

A seleção desses elementos buscou possibilitar a comparação com os dados disponibilizados pela BND-PT, conforme descrito no [notebook](..///BND-PT/escopo.ipynb).


In [4]:
#subfield code="a" do datafield tag="245" corresponde ao Título do periódico
#subfield code="b" do datafield tag="245" corresponde ao Subtítulo do periódico
#subfield code="a" do datafield tag="260" corresponde ao Local de publicação
#subfield code="b" do datafield tag="260" corresponde ao Editora de publicação
#subfield code="c" do datafield tag="260" corresponde ao Período de publicação
#subfield code="a" do datafield tag="301" corresponde à periodicidade
#subfield code="a" do datafield tag="546" corresponde ao idioma da publicação
# criar um csv com os dados selecionados
import csv

with open('./data/complete_data.csv', 'w', newline='', encoding='utf-8') as f:
    writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_ALL)
    writer.writerow(['title', 'subtitle', 'place', 'period', 'editor', 'periodicity', 'language'])
    for record in root:
        title = record.find("./datafield[@tag='245']/subfield[@code='a']")
        subtitle = record.find("./datafield[@tag='245']/subfield[@code='b']")
        place = record.find("./datafield[@tag='260']/subfield[@code='a']")
        period = record.find("./datafield[@tag='260']/subfield[@code='c']")
        editor = record.find("./datafield[@tag='260']/subfield[@code='b']")
        periodicity = record.find("./datafield[@tag='310']/subfield[@code='a']")
        language = record.find("./datafield[@tag='546']/subfield[@code='a']")
        if title is not None:
            title = title.text
        if subtitle is not None:
            subtitle = subtitle.text
        if place is not None:
            place = place.text
        if period is not None:
            period = period.text
        if editor is not None:
            editor = editor.text
        if periodicity is not None:
            periodicity = periodicity.text
        if language is not None:
            language = language.text
        writer.writerow([title, subtitle, place, period, editor, periodicity, language])

## Analisar dados com Pandas

In [5]:
# importar biblioteca
import pandas as pd

In [6]:
# ler csv com pandas
df = pd.read_csv('./data/complete_data.csv', encoding='utf-8')


In [7]:
df.head()

,title,subtitle,place,period,editor,periodicity,language
0,O Abolicionista Paraense,NaN,"Belém, PA",1883-,Typ. da Provincia do Para,Semanal,por
1,O Abolicionista,propriedade de uma associação,"São Luis, MA",1885,Typ. do Abolicionista,NaN,por
2,O Academico,"periodico scientifico, litterario e especialme...","Rio de Janeiro, RJ",1855-,"Typ. Fluminense, de D.L. dos Santos",NaN,por
3,A Actualidade,orgao do Partido Liberal,"Ouro Preto, MG",1878-[1882],Typ. de Jose Egydio da Silca Campos,3 vezes por semana,NaN
4,A Actualidade,"periodico imparcial, litterario, critico e not...",Maranhão,1900-,Typ. de Antonio Pereira Ramos d'Almeida e C. S...,3 vezes por mês,por


In [8]:
# contar número de registros
df.count()

title          7685
subtitle       3666
place          7608
period         7265
editor         5675
periodicity    5901
language       6614
dtype: int64

### Locais de publicação

:warning: ainda é preciso limpar os dados da coluna 'place'

In [9]:
# contar locais de publicação
df['place'].value_counts()

Rio de Janeiro, RJ      2185
São Paulo, SP            294
Florianópolis, SC        239
Recife, PE               207
Bahia [Salvador, BA]     192
                        ... 
Lageado, MT                1
Joinvelle                  1
Tiradentes, MG             1
Duque de Caxias, RJ        1
Urussanga                  1
Name: place, Length: 672, dtype: int64

### Editoras

:warning: ainda é preciso limpar os dados da coluna 'editor'

In [10]:
# contar editor
df['editor'].value_counts()

[s.n.]                                     1764
Typ. do Diário                               44
Typ. Nacional                                42
Typ. Americana                               33
Typ. Cosmopolita                             32
                                           ... 
Typografia do diário                          1
Typ. do Brasil                                1
Diário de Pernambuco                          1
Typ. de J.A. dos Santos Cardoso e Irmao       1
Typ. L. Faria & CIA.                          1
Name: editor, Length: 2020, dtype: int64

### Idiomas

In [11]:
# limpar dados de idioma
# lista de termos para substituir
replace_por = ['Texto em português', 'Texto em portugues', 'Português', 'Em português']
replace_spa = ['Texto em espanhol', 'esp']
replace_fre = ['Texto em frances']

# substituir termos
df['language'] = df['language'].replace(replace_por, 'por')
df['language'] = df['language'].replace(replace_spa, 'spa')
df['language'] = df['language'].replace(replace_fre, 'fre')

In [12]:
# contar idioma
df['language'].value_counts()

por                                              6461
ger                                                61
ita                                                33
fre                                                27
spa                                                17
eng                                                10
ara                                                 2
syr                                                 1
yid                                                 1
Texto em português e alguns textos em francês       1
Name: language, dtype: int64

In [13]:
# porcentagem de idiomas
df['language'].value_counts(normalize=True)


por                                              0.976867
ger                                              0.009223
ita                                              0.004989
fre                                              0.004082
spa                                              0.002570
eng                                              0.001512
ara                                              0.000302
syr                                              0.000151
yid                                              0.000151
Texto em português e alguns textos em francês    0.000151
Name: language, dtype: float64

Interessante notar que noventa e sete porcento dos periódicos estão em português.

[INCLUIR GRÁFICO]

### Período de publicação



In [14]:
#  limpar dados do período de publicação usando regex
# excluir [, ], (, ), :
df['period'] = df['period'].replace(to_replace=r'[\[\]\(\):\?]', value='', regex=True)
#substituir ' - ' por '-'
df['period'] = df['period'].replace(to_replace=r' - ', value='-', regex=True)
#substituir ' a ' por '-'
df['period'] = df['period'].replace(to_replace=r' a ', value='-', regex=True)

In [15]:
# criar coluna com o ano de início da publicação
# se iniciar com dígito, pegar os 4 primeiros caracteres, senão, pegar do 2º ao 5º caractere
df['start_year'] = df['period'].str.extract(r'(\d{4})', expand=False).fillna(df['period'].str[1:5])

In [16]:
# criar coluna com o ano de término da publicação a partir da coluna period
# pegar 4 últimos dígitos que aparecem na coluna period usando regex
df['end_year'] = df['period'].str.extract(r'(\d{4})$', expand=False)

In [17]:
# se 'end_year' for nulo, pegar o ano de início da publicação
df['end_year'] = df['end_year'].fillna(df['start_year'])

In [18]:
#converte as colunas para string
df['start_year'] = df['start_year'].astype(str)
df['end_year'] = df['end_year'].astype(str)

# Substitui todos os caracteres que não são dígitos por uma string vazia
df['start_year'] = df['start_year'].replace(to_replace=r'\D', value='', regex=True)
df['end_year'] = df['end_year'].replace(to_replace=r'\D', value='', regex=True)

# substitui os valores vazios por 0
df['start_year'] = df['start_year'].replace(to_replace=r'^\s*$', value='0', regex=True)
df['end_year'] = df['end_year'].replace(to_replace=r'^\s*$', value='0', regex=True)

# converte as colunas para int
df['start_year'] = df['start_year'].astype(int)
df['end_year'] = df['end_year'].astype(int)


In [19]:
# contar início da publicação
df['start_year'].value_counts()

0       425
1881    168
1890    159
1888    157
1883    144
       ... 
2011      1
87        1
2022      1
1819      1
2017      1
Name: start_year, Length: 214, dtype: int64

In [20]:
# contar fim da publicação
df['end_year'].value_counts()

0       425
1881    163
1890    158
1888    154
1889    143
       ... 
1767      1
87        1
1741      1
1694      1
88        1
Name: end_year, Length: 219, dtype: int64

In [21]:
# contar quando início e fim da publicação são iguais
df[df['start_year'] == df['end_year']].count()

title          5694
subtitle       2874
place          5626
period         5274
editor         4155
periodicity    4507
language       4831
start_year     5694
end_year       5694
dtype: int64

Em 5694 registros, o ano de início da publicação é o mesmo do ano de término, indicando que 7265 periódicos que possuem data registrada no *dataframe*, apenas 1569 possuem data de início e término diferentes. Isso corresponde a uma porcentagem de 21,6% dos periódicos.

In [22]:
# calcular a diferença entre início e fim da publicação
df['diff'] = df['end_year'] - df['start_year']


In [23]:
# listar periódicos com maiores diferenças entre início e fim da publicação
df.sort_values(by='diff', ascending=False).head(20)

,title,subtitle,place,period,editor,periodicity,language,start_year,end_year,diff
5880,O Povo,orgam do Partido Popular,"Pitangui, MG",19924,NaN,Semanal,NaN,1992,9924,7932
4303,O Abaete : jornal noticioso a servico do progr...,NaN,"Abaeté, MG",1948-4949,[s.n.],NaN,por,1948,4949,3001
2489,Diário de Pernambuco,NaN,"Recife, PE",1825-1984,Diário de Pernambuco,Diária,NaN,1825,1984,159
2286,Correio Official de Goyaz,NaN,"Goiás, GO",1837-1943,Typ. Provincial,Desconhecida,por,1837,1943,106
3659,Jornal do Commercio,NaN,"Manaus, AM",1904-2007,Empresa Jornal do Commercio,Diária,NaN,1904,2007,103
3473,Imprensa e Lei,NaN,Lisboa [Portugal],1854-1956,NaN,Indeterminada,por,1854,1956,102
1307,Almanak do Ministerio da Marinha,NaN,"Rio de Janeiro, RJ",1858-1960,NaN,Anual,por,1858,1960,102
6990,Revista do Clube de Engenharia,NaN,"Rio de Janeiro, RJ",1887-1989,O Clube,NaN,por,1887,1989,102
770,A Nova Era,NaN,"Aracaju, SE",1889-1990,[s.n.],Desconhecida,por,1889,1990,101
2597,Documentos,Acta dos festejos civicos com que o povo de So...,"Sobral, CE",1822-1922,NaN,NaN,NaN,1822,1922,100


:warning: Analisar com mais atenção os dados sobre acervos com maior longevidade de publicação.

### Periodicidade

:warning: ainda é preciso limpar os dados da coluna 'frequency'

In [24]:
# contar periodicidade
df['periodicity'].value_counts()

Desconhecida                  2085
Semanal                       1505
Mensal                         425
Indeterminada                  354
Quinzenal                      308
Diária                         258
2 vezes por semana             257
Anual                          170
Irregular                       94
2 vezes por mês                 72
Bissemanal                      61
3 vezes por semana              59
3 vezes por mês                 31
Número único                    29
Duas vezes por semana           28
Trimestral                      28
Bimensal                        26
Bimestral                       17
4 vezes por mes                  8
semanal: nos domingos            8
Varia                            8
Bi-mensal                        7
Trimensal                        7
Semisemanal                      5
Semestral                        5
Três vezes por semana            4
6 vezes por mês                  3
sábados                          3
N. especial         

In [25]:
# contar porcentagem de periodicidade
df['periodicity'].value_counts(normalize=True)

Desconhecida                  0.353330
Semanal                       0.255042
Mensal                        0.072022
Indeterminada                 0.059990
Quinzenal                     0.052195
Diária                        0.043721
2 vezes por semana            0.043552
Anual                         0.028809
Irregular                     0.015930
2 vezes por mês               0.012201
Bissemanal                    0.010337
3 vezes por semana            0.009998
3 vezes por mês               0.005253
Número único                  0.004914
Duas vezes por semana         0.004745
Trimestral                    0.004745
Bimensal                      0.004406
Bimestral                     0.002881
4 vezes por mes               0.001356
semanal: nos domingos         0.001356
Varia                         0.001356
Bi-mensal                     0.001186
Trimensal                     0.001186
Semisemanal                   0.000847
Semestral                     0.000847
Três vezes por semana    